In [ ]:
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import cupy as cp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from skimage.feature import peak_local_max
from scipy.ndimage import center_of_mass
from scipy.spatial import Delaunay
from collections import Counter
import os

In [ ]:
import sys
sys.path.append('D:/GitHubWSU/pfc-blocks/python/pfc-models/ln-vacancy')

import importlib

import PFC2D_Vacancy as PFC2D_Vacancy
importlib.reload(PFC2D_Vacancy)

In [ ]:
outputRoot = "./out/prospectus/vacancy/"

In [ ]:
def Plot(sim, Show=True, file=None, zrange=None):
  if zrange is not None:
     zmin=zrange[0]
     zmax=zrange[1]
  else:
     zmin=sim.phi.get().min()
     zmax=sim.phi.get().max()
  fig = go.Figure(data=go.Heatmap(x=np.arange(sim.nx)*sim.dx,
                                  y=np.arange(sim.ny)*sim.dy,
                                  z=sim.phi.get(),
                                  zmin=zmin,
                                  zmax=zmax,
                                  colorscale='gray',
                                  showscale=False))

  # decide pixels per cell (1 means 1 figure pixel per heatmap cell)
  px_per_cell = 2

  # exact figure pixel size to match heatmap grid
  fig_width = int(sim.nx * px_per_cell)
  fig_height = int(sim.ny * px_per_cell)

  fig.update_layout(
      width=fig_width,
      height=fig_height,
      autosize=False,
      margin=dict(l=0, r=0, t=0, b=0),
      plot_bgcolor='white',
      # place colorbar inside plot or hide it to avoid extra space
      # if you want a colorbar keep it small and place it on top of the image
      # e.g. fig.data[0].colorbar.update({'len':1, 'thickness':8, 'x':0.995})
  )

  fig.update_layout(
      annotations=[
        dict(
          text=f"t = {sim.t:.2f}",
          showarrow=False,
          xref="paper", yref="paper",
          x=0.05, y=0.9,
          xanchor="left", yanchor="top",
          font=dict(color="cyan", size=16, family="Arial"),
        ),
        dict(
          text=f"f = {sim.f:.8f}",
          showarrow=False,
          xref="paper", yref="paper",
          x=0.95, y=0.1,
          xanchor="right", yanchor="bottom",
          font=dict(color="yellow", size=16, family="Arial"),
        )
      ],
  )

  fig.update_xaxes(
      showgrid=False, showticklabels=False,
      # force axis to use full horizontal domain
      domain=[0.0, 1.0],
      range=[0, sim.nx * sim.dx]   # match the heatmap coordinates
  )

  fig.update_yaxes(
      showgrid=False, showticklabels=False,
      domain=[0.0, 1.0],
      range=[0, sim.ny * sim.dy],  # match the heatmap coordinates
      scaleanchor='x',             # lock aspect ratio so a unit in x equals unit in y
      scaleratio=1
  )

  # optionally hide the colorbar to remove its margin impact
  if hasattr(fig.data[0], 'colorbar'):
      fig.data[0].colorbar.update({'thickness': 8, 'len': 1, 'x': 0.995})  # place thin bar on right
      # or remove it: fig.data[0].colorbar = dict() or fig.data[0].showscale = False

  if Show:
    fig.show()

  if file is not None:
    fig.write_image(file)

## 1. Standard PFC

In [ ]:
beta_val = 0.5 # 1.0
epsilon_val = -0.15 #-2.82 -0.1
g_val = 0 #-1
# phi_s = -0.23 # triangle
# phi_s = -0.0 # stripe
# phi_s = 0.23 # honeycomb
phi_s = -0.23 # liquid
phi_l = -0.27
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 0
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
sim.phi[144-17:144+17,148:148+34] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim, Show=True, file=outputRoot + f"vacancy_std_{frame:04d}.png", zrange=None)
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-3)
total_steps = int(4e4)
energy_calcs = 10
frames = 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim, Show=False, file=outputRoot + f"vacancy_std_{frame:04d}.png", zrange=None)
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim)
phi_std = sim.phi.get()

In [ ]:
phi_std.min(), phi_std.max()

In [ ]:
phi = sim.phi.get()
# phi -= phi.mean()
x = sim.x.get()
y = sim.y.get()
r = np.linspace(0, 40, 100)
_corr = np.zeros_like(r)
for _ir in range(0, len(r)):
    _corr_r = 0.0
    _count = 0
    for _ix in range(0, sim.nx):
        for _iy in range(0, sim.ny):
            _rx = ryx[_ix, _iy]
            if (_rx >= r[_ir]) and (_rx < r[_ir] + (r[1]-r[0])):
                _corr_r += phi[_ix, _iy] * phi[(_ix + 50) % sim.nx, (_iy + 50) % sim.ny]
                _count += 1
    if _count > 0:
        _corr[_ir] = _corr_r / _count
    else:
        _corr[_ir] = 0.0

fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=_corr, mode='lines', line_shape='spline', name='Correlation'))
fig.update_layout(title='Spatial Correlation Function',
                  xaxis_title='Distance r',
                  yaxis_title='Correlation C(r)')
fig.show()

In [ ]:
sim.nx * sim.dx / sim.mx

In [ ]:
corr_hat = np.abs(np.fft.fft(_corr))
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.fft.fftfreq(len(_corr))[1:int(len(_corr)/2)]*2*np.pi, y=corr_hat[1:int(len(_corr)/2)], mode='lines', line_shape='spline', name='Correlation'))
fig.update_layout(title='Spatial Correlation Function',
                  xaxis_title='$q$',
                  yaxis_title='$\hat{C}(q)$')
fig.show()

## 2. Log PFC

In [ ]:
beta_val = 0.5 # 1.0
epsilon_val = -2.5 #-2.82 -0.1
g_val = -1
phi_s = 0.50  #-0.1 #-0.18 #0.5 #
phi_l = 0.21
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -10.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 1
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-3
sim.parms.seed = 10
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
sim.phi[144-17:144+17,148:148+34] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim, Show=True, file=outputRoot + f"vacancy_log_{frame:04d}.png", zrange=None)
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-3)
total_steps = int(4e4)
energy_calcs = 10
frames = 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim, Show=False, file=outputRoot + f"vacancy_log_{frame:04d}.png", zrange=None)
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim)
phi_log = sim.phi.get()

In [ ]:
phi = sim.phi.get()
x = sim.x.get()
y = sim.y.get()
r = np.linspace(0, 40, 100)
_corr = np.zeros_like(r)
for _ir in range(0, len(r)):
    _corr_r = 0.0
    _count = 0
    for _ix in range(0, sim.nx):
        for _iy in range(0, sim.ny):
            _rx = ryx[_ix, _iy]
            if (_rx >= r[_ir]) and (_rx < r[_ir] + (r[1]-r[0])):
                _corr_r += phi[_ix, _iy] * phi[(_ix + 50) % sim.nx, (_iy + 50) % sim.ny]
                _count += 1
    if _count > 0:
        _corr[_ir] = _corr_r / _count
    else:
        _corr[_ir] = 0.0

fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=_corr, mode='lines', line_shape='spline', name='Correlation'))
fig.update_layout(title='Spatial Correlation Function',
                  xaxis_title='Distance r',
                  yaxis_title='Correlation C(r)')
fig.show()

## Profile plots

In [ ]:
sim.mx, sim.nx, sim.dx, sim.nx*sim.dx/sim.mx
a = sim.nx*sim.dx/sim.mx

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std1.mean(axis=1), mode='lines', name='t=10', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std.mean(axis=1), mode='lines', name='t=1000', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.05,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
fig.write_image("./out/prospectus/fig-sl-coexistence-stdpfc-profile.png")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log1.mean(axis=1), mode='lines', name='t=1', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log.mean(axis=1), mode='lines', name='t=100', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
fig.write_image("./out/prospectus/fig-sl-coexistence-logpfc-profile.png")

In [ ]:
# Plot phi4 well
fig = go.Figure()
_x = np.linspace(-0.5,0.5,100)
fig.add_trace(go.Scatter(x = _x, y = _x**4 - 0.15*_x**2))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.05,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()

In [ ]:
# Plot phi4 well
fig = go.Figure()
_x = np.linspace(0.001, 2.5, 100)
fig.add_trace(go.Scatter(x = _x, y = _x**4 - _x**3 - 3*_x**2 + _x * np.log(_x)))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=100,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()